In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils.np_utils import to_categorical
import collections
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

def show_image_grids(train_images, train_labels):
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_images[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[train_labels[i]])
    plt.show()

#問題一
def question1(history):
    print(history.history.keys())
    plt.plot(history.history['acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()

#問題3
def question3():
    fashion_mnist = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    train_images = train_images / 255.0
    test_images = test_images / 255.0
    print(train_labels)
    # one-hot
    train_labels = to_categorical(y=train_labels, num_classes=10)
    test_labels = to_categorical(y=test_labels, num_classes=10)
    print(train_labels)
    model = keras.Sequential([keras.layers.Flatten(input_shape=(28,28)),
                            keras.layers.Dense(128, bias_initializer='ones', activation=tf.nn.relu),
                             keras.layers.Dense(10, bias_initializer='ones', activation=tf.nn.softmax)])
    model.summary()
    model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy']) 
    
    model.fit(train_images, train_labels, epochs=20, callbacks = [cp_callback, tb_callback])
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print("test_acc:",test_acc)

if __name__ == '__main__':
    fashion_mnist = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    print(train_images.shape)
    print(train_labels.shape)
    print(test_images.shape)
    print(test_labels.shape)  

    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    #show_image_grids(train_images, train_labels) 
    show_image_grids(train_images, train_labels)

    # Steps 3~8 
    print("Step3~8")
    train_images = train_images / 255.0
    test_images = test_images / 255.0


    model = keras.Sequential([keras.layers.Flatten(input_shape=(28,28)),
                            keras.layers.Dense(128, activation=tf.nn.relu),
                            keras.layers.Dense(10, activation=tf.nn.softmax)])
    model.summary()

    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

    tb_callback     = TensorBoard(log_dir='log_fashion')
    checkpoint_path = "ckpt_fashion/cp-{epoch:04d}.ckpt"
    cp_callback     = ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1, period=5)

    # Steps 9, 10
    print("Step9、10")
    history = model.fit(train_images, train_labels, epochs=20, callbacks = [cp_callback, tb_callback])
    test_loss, test_acc = model.evaluate(test_images, test_labels)
    print("test_acc:",test_acc)

    #問題1
    #question1(history)

    # Step 12
    print("Step12")
    num = ["05","10","15","20"]  
    for i in range(4):
        weight_path = "ckpt_fashion/cp-00"+num[i]+".ckpt"  
        model.load_weights(weight_path)
        loss, acc = model.evaluate(test_images, test_labels)
        print("cp-00"+num[i]+"_acc:",acc)
  

    # Step 13
    print("Step13")
    prob = model.predict(test_images)
    print(prob.shape)
    predictions   = np.zeros(shape=(prob.shape[0],1))
    test_accuracy = np.zeros(shape=(prob.shape[0],1))
    for i in range(prob.shape[0]):      
        predictions[i] = np.argmax(prob[i,:])
        if predictions[i] == test_labels[i]:
            test_accuracy[i] = 1
        else:
            pass
    print("test_accuracy:",np.mean(test_accuracy))

    # Step 14
    print("Step14")
    img = np.zeros((2,28,28))
    img[0] = test_images[4]
    img[1] = np.fliplr(test_images[4])
    prob = model.predict(img)
    pred = np.array([0,0]) 

    for i in range(2):
        pred[i] = np.argmax(prob[i,:])
        plt.grid(False)
        plt.subplot(1,2,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.imshow(img[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[pred[i]]+'{:.0%}'.format(prob[i,pred[i]])+"\n(shirt is the true label)")
    plt.show()


ModuleNotFoundError: No module named 'keras'

In [ ]:
# 1. How many training epochs in model.fit() are required such that the training loss is converged? (epochs=20 其實還未收斂，需畫出 loss vs. epochs 曲線) 
# 500次epochs測試，在epochs=169時acc第一次達到0.99，在epochs=195之後acc都在0.99以上，約epochs=300之後acc都在0.995左右

In [ ]:
# 2. Adjust the batch_size in model.fit(). What will happen if we change batch_size from 32 to 128? (訓練時間是增加或減少？測試正確率是上升或下降？原因為何？)
# batch_size=32訓練時間短正確率下降，batch_size=128訓練時間長正確率上升，神經元多訓練時間較長但是較精準。

In [ ]:
# 3. What is the difference between the following two options in model.compile()? 
#	loss='categorical_crossentropy'
#	loss='sparse_categorical_crossentropy'
# categorical_crossentropy的labels要用 One-Hot-Encoding，實作如question3